# Objektübertragung zwischen Client und Server


Dieses Notebook zeigt, wie man mit **Pydantic** Objekte **senden und empfangen** kann, z. B. in einer Flask-Client-Server-Anwendung.

## Installation

In [ ]:
pip install flask pydantic requests

## 1. Grundlagen von Pydantic

Pydantic ist ein Python-Modul, das beim Arbeiten mit strukturierten Daten hilft, z. B. beim Austausch zwischen Client und Server.  
Es ermöglicht, Klassen mit **Typangaben** zu definieren und überprüft automatisch, ob empfangene Daten diesen Typen entsprechen.  

Pydantic kann Objekte einfach in **JSON** umwandeln (`model_dump()`, `model_dump_json()`), was ideal für Webanwendungen ist.  
Dadurch entstehen **sichere, saubere und gut strukturierte Datenmodelle**, die Fehler im Datenaustausch deutlich reduzieren.

In [ ]:
from pydantic import BaseModel

class Student(BaseModel):
    name: str
    age: int
    course: str

# Beispielobjekt
s = Student(name="Anna", age=21, course="Informatik")
print(s)
print(s.model_dump())  # Als Dictionary


### Erklärung der Klasse `Student`

In diesem Code wird eine **Pydantic-Klasse** namens `Student` definiert.  
Sie erbt von `BaseModel`, was bedeutet, dass sie automatisch alle Funktionen von **Pydantic** nutzt – z. B. Typprüfung und JSON-Umwandlung.  
Die drei Zeilen darin sind **Felddefinitionen** mit Typangaben:  
- `name: str` → der Name muss ein **Text (String)** sein  
- `age: int` → das Alter muss eine **Ganzzahl (Integer)** sein  
- `course: str` → der Kursname ist ebenfalls ein **Text**  

Wenn man ein Objekt `Student(name="Anna", age=20, course="Informatik")` erstellt, überprüft Pydantic automatisch, ob die übergebenen Werte zu diesen Typen passen – und gibt sonst eine Fehlermeldung aus.

### Eine eigene `__init__`-Methode in Pydantic-Klassen

Man **kann** in einer Pydantic-Klasse eine eigene `__init__`-Methode anlegen – aber man sollte **sehr vorsichtig** sein.  
Pydantic erzeugt bereits automatisch eine `__init__`, die alle Felder prüft, validiert und konvertiert.  
Wenn du eine eigene `__init__` schreibst, **überschreibst du diese Logik**, und die automatische **Validierung kann verloren gehen**.  
Deshalb ist es besser, zusätzliche Aktionen in einer **`model_post_init()`**-Methode auszuführen – sie wird **nach der automatischen Initialisierung** aufgerufen.  

### Klassenattribute in Pydantic

In **Pydantic** müssen Klassenattribute mit **`ClassVar`** aus dem Modul `typing` markiert werden,  
damit sie **nicht** als normale Datenfelder (Model-Felder) behandelt und validiert werden.  

Beispiel:

In [82]:
from typing import ClassVar
from pydantic import BaseModel

class Person(BaseModel):

    name: str
    age: int

    # Klassenattribut
    anzahl: ClassVar[int] = 0

    def model_post_init(self, __context):
        Person.anzahl += 1
        print(f"Person {self.name} wurde erstellt!")

# korrekt – Validation bleibt erhalten
s = Person(name="Anna", age=21)
t = Person(name="Berta", age=12)
print(Person.anzahl)

Person Anna wurde erstellt!
Person Berta wurde erstellt!
2


## 2. Attributwerte überprüfen

Mit `Field()` können in **Pydantic** Regeln für einzelne Attribute festgelegt werden.  
Dadurch lassen sich **Eingaben validieren** und **Standardwerte setzen**.

### Tabelle: Wichtige Optionen von `Field()`

| Option | Bedeutung | Beispiel | Wirkung |
|:--------|:-----------|:----------|:---------|
| `default` | Standardwert für das Feld | `Field(default=0)` | Wenn kein Wert übergeben wird, wird 0 gesetzt |
| `ge` | „greater or equal“ – Minimalwert | `Field(..., ge=0)` | Wert muss ≥ 0 sein |
| `gt` | „greater than“ – strenger Minimalwert | `Field(..., gt=0)` | Wert muss > 0 sein |
| `le` | „less or equal“ – Maximalwert | `Field(..., le=100)` | Wert muss ≤ 100 sein |
| `lt` | „less than“ – strenger Maximalwert | `Field(..., lt=100)` | Wert muss < 100 sein |
| `min_length` | Minimale Länge von Strings oder Listen | `Field(..., min_length=3)` | Prüft, dass Text oder Liste mindestens 3 Elemente hat |
| `max_length` | Maximale Länge von Strings oder Listen | `Field(..., max_length=20)` | Prüft, dass Text oder Liste höchstens 20 Elemente hat |


Der folgende Code zeigt, wie man mit **Pydantic** ein Datenmodell erstellt, das **automatisch überprüft**, ob eingegebene Werte **gültig** sind – sowohl beim Erstellen des Objekts als auch bei späteren Änderungen.
Das ist z. B. sinnvoll, wenn man Daten (wie PS oder Verbrauch) sicher speichern will, ohne dass ungültige Werte erlaubt sind.

Eine vollständige Beschreibung findet ihr unter https://docs.pydantic.dev/latest/concepts/models/


In [ ]:
from pydantic import BaseModel, Field, field_validator, ValidationError

class Auto(BaseModel):

    # Eingabefelder mit Pydantic-Validierung
    ps: int = Field(default=75, ge=0, le=500)
    verbrauch: float = Field(default=5.0)

    # alternativ: Individuelle Prüfung nur für 'verbrauch'
    @field_validator("verbrauch")
    def check_verbrauch(cls, value):
        if not 0 <= value <= 20:
            raise ValueError("Verbrauch muss zwischen 0 und 20 Litern liegen")
        return value

    # Aktiviert Live-Validierung bei späteren Änderungen
    model_config = {"validate_assignment": True}


# Objekt erzeugen (gültige Werte)
a = Auto(ps=120, verbrauch=6.5)
print("1️⃣ Neues Auto:", a.model_dump())

# Änderung eines gültigen Wertes
a.ps = 90
print("2️⃣ Nach Änderung:", a.model_dump())

# Ungültiger Wert für PS
try:
    a.ps = -50
    print(a.model_dump())
except ValueError as e:
    print("Fehler:", e)

# Ungültiger Wert für Verbrauch
try:
    a.verbrauch = 30
    print(a.model_dump())
except ValueError as e:
    print("Fehler:", e)

# Fehler schon beim Erstellen (beide Werte falsch)
try:
    a = Auto(ps=-300, verbrauch=-5)
except ValidationError as e:
    print("Fehler beim Erstellen des Objekts:")
    print(e)

1️⃣ Neues Auto: {'ps': 120, 'verbrauch': 6.5}
2️⃣ Nach Änderung: {'ps': 90, 'verbrauch': 6.5}
Fehler: 1 validation error for Auto
ps
  Input should be greater than or equal to 0 [type=greater_than_equal, input_value=-50, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than_equal
Fehler: 1 validation error for Auto
verbrauch
  Value error, Verbrauch muss zwischen 0 und 20 Litern liegen [type=value_error, input_value=30, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error
Fehler beim Erstellen des Objekts:
2 validation errors for Auto
ps
  Input should be greater than or equal to 0 [type=greater_than_equal, input_value=-300, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than_equal
verbrauch
  Value error, Verbrauch muss zwischen 0 und 20 Litern liegen [type=value_error, input_value=-5, input_type=int]
    For further information visit https://errors.pyd

# Aufgabe
Vergleiche die Validierung mittels `Field` mit der Validierung über Datenkapselung in Python, die wir bisher genutzt haben. 

## 3. Objekt senden (Server → Client)

In [ ]:

from flask import Flask, jsonify

app = Flask(__name__)

@app.route("/student", methods=["GET"])
def send_student():
    student = Student(name="Tom", age=20, course="IT")
    return jsonify(student.model_dump())



## 4. Objekt empfangen (Client → Server)

In [ ]:

from flask import request

@app.route("/student", methods=["POST"])
def receive_student():
    data = request.get_json()
    student = Student(**data)  # JSON → Objekt
    print("Empfangen:", student)
    return jsonify({"message": "Student empfangen", "student": student.model_dump()})


### Bedeutung von `**data`

Der Operator `**` entpackt ein **Dictionary** in einzelne **Schlüssel-Wert-Paare** und übergibt sie als **benannte Argumente** an die Funktion oder den Konstruktor.  
Im Beispiel `Student(**data)` bedeutet das: wenn `data = {"name": "Anna", "age": 20}`, wird intern aufgerufen wie `Student(name="Anna", age=20)`.

## 5. Client-Simulation mit requests

In [ ]:

import requests

data = {"name": "Lena", "age": 22, "course": "Medientechnik"}
# response = requests.post("http://127.0.0.1:5000/student", json=data)
# print(response.json())
print("Dies wäre der POST-Aufruf zum Server – bitte lokal testen.")


## <font color=red >Übung</font> 
1. Erstelle das Modul `model`
   - Lege dort die Definition deiner Klasse `Spieler` ab.
   - Erweitere deine Klasse Spieler um die Validierung per Pydantic.
   - Überprüfe deinen Code an Fehlerbeispielen und korrekten Beispielen.